### Train an MMDetection Network
- See [tutorial](https://github.com/open-mmlab/mmdetection/blob/main/demo/MMDet_Tutorial.ipynb)

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from pathlib import Path
import sys
from datetime import datetime

### Download checkpoint for a pretrained model (if desired)
Alternatively, use a previous mouse model as a pretrained model

In [3]:
pretrained_model_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models")
pretrained_model_directory.mkdir(parents=True, exist_ok=True)

In [4]:
# find models here: https://github.com/open-mmlab/mmdetection/tree/main/configs/rtmdet
pretrain_model = "rtmdet_s_8xb32-300e_coco"

In [5]:
command = f"source activate {Path(sys.executable).parents[1]}; mim download mmdet --config {pretrain_model} --dest {pretrained_model_directory.as_posix()}"
print(command)

source activate /n/groups/datta/tim_sainburg/conda_envs/mmdeploy; mim download mmdet --config rtmdet_s_8xb32-300e_coco --dest /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models


In [6]:
!{command}

/bin/bash: activate: No such file or directory
processing rtmdet_s_8xb32-300e_coco...
rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth exists in /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models
Traceback (most recent call last):
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(sel

In [7]:
!ls {pretrained_model_directory.as_posix()}

rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth
rtmdet_s_8xb32-300e_coco.py
rtmpose-m_8xb64-210e_ap10k-256x256.py
rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth


In [8]:
pretrained_model_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models')

### Parameters and dataset

In [9]:
model_name = 'rtmdet_small_8xb32-300e_coco_rat'

# Where the COCO format dataset is located (created in the previous notebook)
dataset_directory = Path("/n/groups/datta/jlove/data/rat_seq/6cam/detection_training/08-26-24-rat_6cam_alldata")

# which config to use (this is what we base the config off of). Should be in the mmdeteciton repo. 
config_loc = Path('/n/groups/datta/tim_sainburg/projects/mmdetection/configs/rtmdet/rtmdet_s_8xb32-300e_coco.py')

# which pretrained model to use (point to .pth file). Pretrained model should be the same model architecture. 
#pretrained_model = pretrained_model_directory / "rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth"
pretrained_model = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_24-01-05-11-25-00_102726/epoch_300.pth')
use_pretrained_model = True

# working directory (where model output is saved)
output_directory = Path("/n/groups/datta/jlove/data/rat_seq/6cam/mm_training")
formatted_datetime = datetime.now().strftime("%y-%m-%d-%H-%M-%S")
working_directory = (output_directory / 'rtmdet' / f"{model_name}_{formatted_datetime}")
working_directory.mkdir(parents=True, exist_ok=True)

# You shouldn't need to change anything below here

### Display compute / environment info (for future reference)

In [10]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [11]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env
import sys
import mmdet
import torch, torchvision
import mmpose
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info

print(f"Environment: {sys.executable}")
for name, val in collect_env().items():
    print(f'{name}: {val}')
# Check Pytorch installation
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)
print('mmpose version:', mmpose.__version__) 

Environment: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3
sys.platform: linux
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
CUDA available: True
numpy_random_seed: 2147483648
GPU 0: Quadro RTX 6000
CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy
NVCC: Cuda compilation tools, release 12.1, V12.1.105
GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
PyTorch: 2.1.1
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arc

### Create the config file

In [12]:
from mmengine import Config
from pathlib import Path

In [13]:
cfg = Config.fromfile(config_loc.as_posix())

# set the dataset directory
cfg.data_root = dataset_directory.as_posix()

# set the working directory
cfg.work_dir = working_directory.as_posix()

# set head to only care about the mouse class
cfg.model.bbox_head.num_classes = 1

# set the metainfo
cfg.metainfo = {
    'classes': ('Mouse', ),
    'palette': [
        (220, 20, 60),
    ]
}

# specify the dataset
cfg.dataset_type = 'CocoDataset'

# load COCO pre-trained weight
if use_pretrained_model:
    cfg.load_from = pretrained_model.as_posix()

In [14]:
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.metainfo = cfg.metainfo
cfg.train_dataloader.dataset.data_prefix = dict(img='train/')
cfg.train_dataloader.dataset.ann_file = 'annotations/instances_train.json'


cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.metainfo = cfg.metainfo
cfg.val_dataloader.dataset.data_prefix = dict(img='val/')
cfg.val_dataloader.dataset.ann_file = 'annotations/instances_val.json'

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.type = cfg.dataset_type

cfg.val_evaluator.ann_file= cfg.data_root + '/annotations/instances_val.json'
cfg.test_evaluator.ann_file= cfg.data_root + '/annotations/instances_val.json'

cfg.default_hooks.checkpoint.max_keep_ckpts = 15
cfg.default_hooks.checkpoint.interval = 50

cfg.max_epochs = 2000
cfg.train_cfg.max_epochs = 2000

In [15]:
print(cfg.model)

{'type': 'RTMDet', 'data_preprocessor': {'type': 'DetDataPreprocessor', 'mean': [103.53, 116.28, 123.675], 'std': [57.375, 57.12, 58.395], 'bgr_to_rgb': False, 'batch_augments': None}, 'backbone': {'type': 'CSPNeXt', 'arch': 'P5', 'expand_ratio': 0.5, 'deepen_factor': 0.33, 'widen_factor': 0.5, 'channel_attention': True, 'norm_cfg': {'type': 'SyncBN'}, 'act_cfg': {'type': 'SiLU', 'inplace': True}, 'init_cfg': {'type': 'Pretrained', 'prefix': 'backbone.', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v3.0/rtmdet/cspnext_rsb_pretrain/cspnext-s_imagenet_600e.pth'}}, 'neck': {'type': 'CSPNeXtPAFPN', 'in_channels': [128, 256, 512], 'out_channels': 128, 'num_csp_blocks': 1, 'expand_ratio': 0.5, 'norm_cfg': {'type': 'SyncBN'}, 'act_cfg': {'type': 'SiLU', 'inplace': True}}, 'bbox_head': {'type': 'RTMDetSepBNHead', 'num_classes': 1, 'in_channels': 128, 'stacked_convs': 2, 'feat_channels': 128, 'anchor_generator': {'type': 'MlvlPointGenerator', 'offset': 0, 'strides': [8, 16, 32]}, '

In [16]:
# save configuration file for future reference
cfg.dump(working_directory / 'config.py')

In [17]:
print(working_directory)

/n/groups/datta/jlove/data/rat_seq/6cam/mm_training/rtmdet/rtmdet_small_8xb32-300e_coco_rat_24-08-26-23-29-19


### Train

In [18]:
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

In [19]:
# build the runner from config
runner = Runner.from_cfg(cfg)

08/26 23:29:32 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 306267717
    GPU 0: Quadro RTX 6000
    CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
    PyTorch: 2.1.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-g

In [20]:
cfg

Config (path: /n/groups/datta/tim_sainburg/projects/mmdetection/configs/rtmdet/rtmdet_s_8xb32-300e_coco.py): {'default_scope': 'mmdet', 'default_hooks': {'timer': {'type': 'IterTimerHook'}, 'logger': {'type': 'LoggerHook', 'interval': 50}, 'param_scheduler': {'type': 'ParamSchedulerHook'}, 'checkpoint': {'type': 'CheckpointHook', 'interval': 50, 'max_keep_ckpts': 15}, 'sampler_seed': {'type': 'DistSamplerSeedHook'}, 'visualization': {'type': 'DetVisualizationHook'}}, 'env_cfg': {'cudnn_benchmark': False, 'mp_cfg': {'mp_start_method': 'fork', 'opencv_num_threads': 0}, 'dist_cfg': {'backend': 'nccl'}}, 'vis_backends': [{'type': 'LocalVisBackend'}], 'visualizer': {'type': 'DetLocalVisualizer', 'vis_backends': [{'type': 'LocalVisBackend'}], 'name': 'visualizer'}, 'log_processor': {'type': 'LogProcessor', 'window_size': 50, 'by_epoch': True}, 'log_level': 'INFO', 'load_from': '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco

In [21]:
!ls /n/groups/datta/jlove/data/rat_seq/6cam/detection_training/08-26-24-rat_6cam_alldata

annotations  calib_params  train  val


In [ ]:
# start training
runner.train()

loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.0.bn.weight:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.0.bn.bias:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.1.bn.weight:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.1.bn.bias:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.2.bn.weight:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stem.2.bn.bias:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stage1.0.bn.weight:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stage1.0.bn.bias:weight_decay=0.0
08/26 23:29:39 - mmengine - INFO - paramwise_options -- backbone.stage1.1.main_conv.bn.weight:weight_decay=0.0
08/26 23:29:39 - m

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
08/26 23:29:41 - mmengine - INFO - load backbone. in model from: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/cspnext_rsb_pretrain/cspnext-s_imagenet_600e.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection/v3.0/rtmdet/cspnext_rsb_pretrain/cspnext-s_imagenet_600e.pth
Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_24-01-05-11-25-00_102726/epoch_300.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.stem.0.conv.weight: copying a param with shape torch.Size([12, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 3, 3, 3]).
size mismatch for backbone.stem.0.bn.weight: copying a param with shape torch.Size([12]) 

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201336/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
aa